## **0 Prepare Environment**


In [1]:
!python --version

Python 3.8.16


In [2]:
!nvidia-smi

Sun Jan  8 09:24:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install https://github.com/mlc-ai/utils/releases/download/v0.9.dev0/mlc_ai_nightly_cu111-0.9.dev2972+g78908c2ea-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.4/307.4 MB 1.7 MB/s eta 0:00:00


Lots of API changes between mlc.ai and tvm repo, so the jupyter notebook provided in the course will not work. 

tune_tir API reference:

https://github.com/apache/tvm/blob/main/tests/python/unittest/test_meta_schedule_tune_tir.py

## **1 Auto Schedule**

In [4]:
import tvm
import tvm.testing
from tvm import te # tensor expression
import numpy as np

from tvm.script import tir as T
from tvm import meta_schedule as ms

In [5]:
M = 1024
N = 1024
K = 1024

In [6]:
# define computation using tvm script
@tvm.script.ir_module
class MyMatMultModule:
  @T.prim_func
  def main(A: T.Buffer[(M, K), "float32"],
           B: T.Buffer[(K, N), "float32"],
           C: T.Buffer[(M, N), "float32"],
           ):
    T.func_attr({"global_symbol": "main", "tir.noalias": True})
    for i, j, k in T.grid(M, N, K):
      with T.block("C"):
        vi, vj, vk = T.axis.remap("SSR", [i, j, k])
        with T.init():
          C[vi, vj] = 0.0
        C[vi, vj] = C[vi, vj] + A[vi, vk] * B[vk, vj]

In [31]:
database = ms.tune_tir(
    mod=MyMatMultModule,
    target="nvidia/nvidia-t4", # define target type
    work_dir="./tune_tmp",
    max_trials_global=64,
    num_trials_per_iter=64,
    task_name="main"
)

Streaming output truncated to the last 5000 lines.
2023-01-08 10:04:51 [DEBUG] XGB stopped. Best iteration: [11] tr-p-rmse:0.055201	tr-a-peak@32:1	tr-rmse:0.462189	tr-rmse:0.462189 
2023-01-08 10:04:51 [DEBUG] XGB stopped. Best iteration: [11] tr-p-rmse:0.055201	tr-a-peak@32:1	tr-rmse:0.462189	tr-rmse:0.462189 
2023-01-08 10:04:51 [DEBUG] XGB stopped. Best iteration: [11] tr-p-rmse:0.055201	tr-a-peak@32:1	tr-rmse:0.462189	tr-rmse:0.462189 
2023-01-08 10:04:51 [DEBUG] XGB stopped. Best iteration: [11] tr-p-rmse:0.055201	tr-a-peak@32:1	tr-rmse:0.462189	tr-rmse:0.462189 
2023-01-08 10:04:51 [DEBUG] XGB stopped. Best iteration: [11] tr-p-rmse:0.055201	tr-a-peak@32:1	tr-rmse:0.462189	tr-rmse:0.462189 
2023-01-08 10:04:51 [DEBUG] XGB stopped. Best iteration: [11] tr-p-rmse:0.055201	tr-a-peak@32:1	tr-rmse:0.462189	tr-rmse:0.462189 
2023-01-08 10:04:51 [DEBUG] XGB stopped. Best iteration: [11] tr-p-rmse:0.055201	tr-a-peak@32:1	tr-rmse:0.462189	tr-rmse:0.462189 
2023-01-08 10:04:51 [DEBUG] XGB 

In [32]:
# sch_tuned.mod.show() -> this will not work, tune_tir return json_database
sch_tuned = ms.tir_integration.compile_tir(database, MyMatMultModule, "nvidia/nvidia-t4")

In [33]:
sch_tuned.mod.show()

In [34]:
from tvm.script.parser.tir import evaluate
num_flop = 2 * M * N * K
rt_mod = tvm.build(sch_tuned.mod, target="nvidia/nvidia-t4")
dev = tvm.cuda(0)
A_np = np.random.uniform(size=(M, K)).astype("float32")
B_np = np.random.uniform(size=(K, N)).astype("float32")
A_nd = tvm.nd.array(A_np, dev)
B_nd = tvm.nd.array(B_np, dev)
C_nd = tvm.nd.array(np.zeros((M, N), dtype="float32"), dev)
evaluator = rt_mod.time_evaluator("main", dev, number=10)
print("MetaSchedule: %f GFLOPS" % (num_flop / evaluator(A_nd, B_nd, C_nd).mean / 1e9))

MetaSchedule: 1728.331466 GFLOPS


In [11]:
# tvm.lower(sch_tuned.mod, [A_nd, B_nd, C_nd], simple_mode=True)

In [43]:
# print(rt_mod.imported_modules[0].get_source())
with open('matmul_tvm.h', 'w') as f:
  f.write(rt_mod.imported_modules[0].get_source())

In [44]:
ls

main*  main.cu  matmul_tvm.h  sample_data/  tune_tmp/


In [45]:
!nvcc main.cu -o main -std=c++11

In [46]:
ls

main*  main.cu  matmul_tvm.h  sample_data/  tune_tmp/


In [47]:
!./main

Run kernel 1000 times taken 9261924 ns
Precision is accurate


## **Manual Tuning**

In [48]:
def blocking(sch, 
             tile_local_y, 
             tile_local_x, 
             tile_block_y, 
             tile_block_x,
             tile_k):
    block_C = sch.get_block("C")
    C_local = sch.cache_write(block_C, 0, "local")

    i, j, k = sch.get_loops(block=block_C)

    i0, i1, i2 = sch.split(loop=i, factors=[None, tile_block_y, tile_local_y])
    j0, j1, j2 = sch.split(loop=j, factors=[None, tile_block_x, tile_local_x])
    k0, k1 = sch.split(loop=k, factors=[None, tile_k])
    sch.unroll(k1)
    sch.reorder(i0, j0, i1, j1, k0, k1, i2, j2)
    sch.reverse_compute_at(C_local, j1)

    sch.bind(i0, "blockIdx.y")
    sch.bind(j0, "blockIdx.x")

    sch.bind(i1, "threadIdx.y")
    sch.bind(j1, "threadIdx.x")
    sch.decompose_reduction(block_C, k0)

    return sch

sch = tvm.tir.Schedule(MyMatMultModule)
sch = blocking(sch, 8, 8, 8, 8, 4)
sch.mod.show()

In [49]:
rt_mod_manual = tvm.build(sch.mod, target="cuda")
dev = tvm.cuda(0)
A_np = np.random.uniform(size=(1024, 1024)).astype("float32")
B_np = np.random.uniform(size=(1024, 1024)).astype("float32")
A_nd = tvm.nd.array(A_np, dev)
B_nd = tvm.nd.array(B_np, dev)
C_nd = tvm.nd.array(np.zeros((1024, 1024), dtype="float32"), dev)

num_flop = 2 * 1024 * 1024 * 1024
evaluator = rt_mod_manual.time_evaluator("main", dev, number=10)


print("GEMM-Blocking: %f GFLOPS" % (num_flop / evaluator(A_nd, B_nd, C_nd).mean / 1e9))
     

GEMM-Blocking: 865.890694 GFLOPS


In [50]:
def cache_read_and_coop_fetch(sch, block, nthread, read_idx, read_loc):
    read_cache = sch.cache_read(block=block, read_buffer_index=read_idx, storage_scope="shared")
    sch.compute_at(block=read_cache, loop=read_loc)
    # vectorized cooperative fetch
    inner0, inner1 = sch.get_loops(block=read_cache)[-2:]
    inner = sch.fuse(inner0, inner1)
    _, tx, vec = sch.split(loop=inner, factors=[None, nthread, 4])
    sch.vectorize(vec)
    sch.bind(tx, "threadIdx.x")


def blocking_with_shared(
    sch, 
    tile_local_y, 
    tile_local_x, 
    tile_block_y, 
    tile_block_x,
    tile_k):
    block_C = sch.get_block("C")
    C_local = sch.cache_write(block_C, 0, "local")

    i, j, k = sch.get_loops(block=block_C)

    i0, i1, i2 = sch.split(loop=i, factors=[None, tile_block_y, tile_local_y])
    j0, j1, j2 = sch.split(loop=j, factors=[None, tile_block_x, tile_local_x])
    k0, k1 = sch.split(loop=k, factors=[None, tile_k])

    sch.reorder(i0, j0, i1, j1, k0, k1, i2, j2)
    sch.reverse_compute_at(C_local, j1)

    sch.bind(i0, "blockIdx.y")
    sch.bind(j0, "blockIdx.x")

    tx = sch.fuse(i1, j1)
    sch.bind(tx, "threadIdx.x")
    nthread = tile_block_y * tile_block_x
    cache_read_and_coop_fetch(sch, block_C, nthread, 0, k0)
    cache_read_and_coop_fetch(sch, block_C, nthread, 1, k0)    
    sch.decompose_reduction(block_C, k0)

    return sch

sch = tvm.tir.Schedule(MyMatMultModule)
sch = blocking_with_shared(sch, 8, 8, 8, 8, 8)
sch.mod.show()


In [51]:
rt_mod_manual = tvm.build(sch.mod, target="cuda")
dev = tvm.cuda(0)
evaluator = rt_mod_manual.time_evaluator("main", dev, number=10)

print("GEMM-Blocking: %f GFLOPS" % (num_flop / evaluator(A_nd, B_nd, C_nd).mean / 1e9))
     

GEMM-Blocking: 1332.156879 GFLOPS


In [52]:
with open('matmul_tvm_manual.h', 'w') as f:
  f.write(rt_mod_manual.imported_modules[0].get_source())

In [54]:
!nvcc main.cu -o main -std=c++11

In [55]:
!./main

Run kernel 1000 times taken 11239990 ns
Precision is accurate
